In [1]:
import os
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import re

Using TensorFlow backend.


In [2]:
BASE_DIR = "input/"
columns = ['Text', 'Class']
base = pd.DataFrame(columns=columns)

In [3]:
dfs = []
for file in os.listdir(BASE_DIR):
    path = os.path.join(BASE_DIR, file)
    df = pd.read_csv(path, sep="\t", header=None, names=columns)
    dfs.append(df)
base = pd.concat(dfs, ignore_index=True)

In [4]:
def clean_text(text):
    text = re.sub("[^a-zA-Z]"," ",text)
    return text.lower().split() 

In [5]:
max_fatures = 500
base.Text = base.Text.apply(clean_text)
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(base.Text.values)
X = tokenizer.texts_to_sequences(base.Text.values)
X = pad_sequences(X)

In [6]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 972, 128)          64000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 972, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 319,194
Trainable params: 319,194
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
y = pd.get_dummies(base['Class']).values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [9]:
batch_size = 32
model.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
 - 171s - loss: 0.6639 - acc: 0.6110
Epoch 2/10
 - 179s - loss: 0.4961 - acc: 0.7662
Epoch 3/10
 - 162s - loss: 0.4165 - acc: 0.8085
Epoch 4/10
 - 154s - loss: 0.3773 - acc: 0.8339
Epoch 5/10
 - 172s - loss: 0.3542 - acc: 0.8444
Epoch 6/10
 - 156s - loss: 0.3265 - acc: 0.8612
Epoch 7/10
 - 174s - loss: 0.3050 - acc: 0.8653
Epoch 8/10
 - 189s - loss: 0.2945 - acc: 0.8699
Epoch 9/10
 - 198s - loss: 0.2752 - acc: 0.8840
Epoch 10/10
 - 169s - loss: 0.2588 - acc: 0.8899


In [11]:
_, acc = model.evaluate(X_test, y_test, verbose=2)

In [12]:
acc

0.8018181818181818